In [ ]:
#modules
import csv
import datetime
import warnings
import numpy as np
import pandas as pd
import statsmodels.api as sm   
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from itertools import product    
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from matplotlib import dates as mpl_dates

In [ ]:
#preprocessing
def plot_series(time, series, col = 'dodgerblue', lab = 'original', format="-", start=0, end=None):
    plt.style.use('seaborn')
    plt.plot(time[start:end], series[start:end], format, color = col, label = lab)
    # display the grid
    plt.grid(True)
    plt.gcf().autofmt_xdate() 
    # set the format to out x-axis, gca is the get current axis
    plt.gca().xaxis.set_major_formatter(date_formate)
    plt.tight_layout() 
    plt.legend(loc = 'best')
    
    
df = pd.read_csv('file name here')
del df['Unnamed: 0']
df.head()

#results - univariate time series problems, no obvious trend, no white noise, 11 year seasonality

In [ ]:
split_time = int(len(time_step)*0.8)
time_train = time_step[:split_time]
x_train = sunspots[:split_time]
time_valid = time_step[split_time:]
x_valid = sunspots[split_time:]

In [ ]:
split_time = int(len(time_step)*0.8)
train = timeseries[:split_time]
valid = timeseries[split_time:]

In [ ]:
#training
#NOTE: Statistics and other methods  were done and removed -- ARIMA, Moving average and naive forecast were used

#simple DNN setup with windowed dataset to train data
window_size = 60
batch_size = 32
shuffle_buffer_size = 1000

# turn a series into a dataset which we can train on
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    # create a dataset from series
    dataset = tf.data.Dataset.from_tensor_slices(series)
    # slice the data up into appropriate windows
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    # flattened into chunks in the size of our window size + 1
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    # batched into the selected batch size and returned
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
# three layer of 20,10 and 1 neurons, input shape is the size of window 
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(20, input_shape=[window_size], activation="relu"), 
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1)
])

#MAE used to penalize large errors instead of MSE (need to check with both)
model.compile(loss="mae", optimizer=tf.keras.optimizers.SGD(lr=1e-7, momentum=0.9))
# ignore the epoch by epoch output by setting verbose = 0
history = model.fit(dataset,epochs=200,verbose=0)
model.summary()

loss = history.history['loss']
epochs = range(len(loss))
plt.plot(epochs, loss, 'b', label='Training Loss')
plt.show()